In [1]:
%load_ext autoreload
%autoreload 2

In [130]:
from src.driver.service import DriverService
from src.state_manager.state import StateManager
from src.actions.browser_actions import BrowserActions
from src.agent_interface.planing_agent import PlaningAgent
from src.actions.browser_actions import ActionResult
from src.state_manager.utils import save_formatted_html

import os
from datetime import datetime

In [131]:
driver = DriverService().get_driver()
state_manager = StateManager(driver)
actions = BrowserActions(driver)

In [132]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
run_folder = f'temp/run_{timestamp}'
if not os.path.exists(run_folder):
    os.makedirs(run_folder)

In [133]:
task = 'Go to kayak.com and find a flight from Zürich to (ask the user for the destination) on 2025-04-25 with return on 2025-06-05 for 2 people.'

In [134]:
print(actions.get_default_actions())

{'ask_user': 'text: string', 'send_user_text': 'text: string', 'search_google': 'text: string', 'go_to_url': 'url: string', 'done': '', 'go_back': '', 'click': 'id: int', 'input': 'id: int, text: string', 'nothing': ''}


In [135]:
agent = PlaningAgent(default_actions=actions.get_default_actions(), model="gpt-4o")

In [136]:
agent.add_user_prompt(f'Your task is: {task}', after_system=True)

In [137]:
url_history = []
output = ActionResult()

In [138]:
i = 0

In [139]:
previous_state = None

### Repeat the following block!

---

In [209]:
current_state = state_manager.get_current_state(run_folder=run_folder, step=i)

In [210]:
state_manager.add_change_info(current_state=current_state, previous_state=previous_state)

In [211]:
save_formatted_html(driver.page_source, f'{run_folder}/html_{i}.html')

In [212]:
url_history.append(driver.current_url)

In [213]:
elem_text = current_state.dom_content.dom_items_to_string()

In [214]:
text = f'Elements:\n{elem_text}\nUrl history: {url_history}'

In [215]:
print(text)

Elements:
      0:<div role="button" aria-label="Open main navigation"></div>
      1:<a href="/" aria-label="Go to the kayak homepage"></a>
      2:<div role="button" aria-label="Open Trips drawer"></div>
      3:<div role="button">Sign in</div>
      _:Flights
      _:Stays
      _:Cars
      _:Packages
      _:KAYAK.ai
      _:BETA
      _:Plan your trip
      _:Explore
      _:Flight Tracker
      _:Travel tips
      _:KAYAK for Business
      _:NEW
      _:Trips
      _:English
      _:$
      _:United States dollar
      _:Feedback
     21:<div role="combobox" aria-controls="flight-trip-type-dropdown" aria-haspopup="listbox" aria-expanded="false" aria-label="Trip type Round-trip">Round-trip</div>
     22:<div role="button">0 bags</div>
     23:<div role="list">Islamabad (ISB)</div>
     24:	<div role="listitem">Islamabad (ISB)</div>
     25:		<div role="button" aria-label="Remove value"></div>
     26:<input type="text" aria-readonly="false" placeholder="From?" aria-label="Flight

In [216]:
if output.user_input:
    agent.add_user_prompt(output.user_input)
if output.error:
    text += f', Previous action error: {output.error}'
action = agent.chat(
    text, 
    store_conversation=f'{run_folder}/conversation_{i}.txt',
    image=current_state.screenshot
)

Text $0.0053 + Image $0.0002 = $0.0060 for 2180  tokens


In [217]:
action

Action(valuation_previous_goal='Failed to select the Bali destination option.', goal="Select 'Denpasar, Bali, Indonesia' as the destination.", action='click', params=ActionParams(url=None, id=42, text=None))

In [218]:
output = actions.execute_action(action, current_state.dom_content.selector_map)

{
    "valuation_previous_goal": "Failed to select the Bali destination option.",
    "goal": "Select 'Denpasar, Bali, Indonesia' as the destination.",
    "action": "click",
    "params": {
        "url": null,
        "id": 42,
        "text": null
    }
}


In [219]:
if output.done:
    print('Task completed')
    driver.quit()

In [220]:
i += 1

In [221]:
previous_state = current_state

---

In [222]:
driver.quit()